In [1]:
include("./param_discharge.jl");
include("./aux_functions_discharge.jl");

using Main.aux_functions
using Main.param
using ADCME
using SparseArrays

using DelimitedFiles
using Dates
using Plots

# definition of the parameters from the shell file
maxiter = 100
N_steps = 50
print("maximum iteration: ", maxiter, "\t", "Number of steps: ", N_steps, "\n")

# Definition of the parameters of initial descrite probabilities and initial lambda
global N_k_dis_ = 2
global T_exp = -3.0

# Initialization of the model and the optimization process


maximum iteration: 100	Number of steps: 50


2022-05-11 16:08:25.475962: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_100.dll


-3.0

In [2]:
param_model_val = param_model(N_steps = N_steps);


In [3]:
tf_variables, h_t, q_t_x, q_t_y = Darcy_flow_solver(param_model_val);

In [4]:
global model_param = param_model_val

# function P(h_, i, j)
#     # get the head for specifi i and j
#     tf.strided_slice(h_, [0, i + (j + 1) * model_param.N_y], [model_param.N_k, (j - 1) * model_param.N_y - 1], [1, model_param.N_y])
# end

# function P_y(h_, i, j)
#     # get the head for specifi i and j
#     tf.strided_slice(h_, [0, (i - 1) * model_param.N_x + 1 + j] , [model_param.N_k, i * model_param.N_x + j - 1], [1, 1])
# end

param_model
  exp_name: String "experiment11_05_16_09_2"
  run_name: String "Run_no_zero_flow_no_oscilations_Run_Run_1646968123"
  N_points: Int64 9
  Lx: Int64 1000
  Ly: Int64 1000
  T_h: Int64 18000
  T: Int64 18000
  N_steps_orig: Int64 1800
  N_steps: Int64 50
  N_k: Int64 64
  N_x: Int64 22
  N_y: Int64 22
  Ne: Int64 484
  N_steps_fine: Int64 50
  N_k_fine: Int64 100
  n_step_skip: Int64 36
  dt: Float64 360.0
  dx: Float64 47.61904761904762
  dy: Float64 47.61904761904762
  loc_x_list: Array{Int64}((9,)) [6, 11, 16, 6, 11, 16, 6, 11, 16]
  loc_y_list: Array{Int64}((9,)) [6, 6, 6, 11, 11, 11, 16, 16, 16]
  K_mupltiplier: Float64 1.0


In [5]:
loss, dw_2, opt_ADAM, opt_LFGS, opt_ADAM_sum, opt_LFGS_sum, diff_eval, p_pre_soft_max, p, opt_LFGS_x = Info_upscale(tf_variables, param_model_val, q_t_x, q_t_y, N_k_dis_, maxiter)


C:\Users\dbeli\.julia\adcme\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\dbeli\.julia\adcme\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\dbeli\.julia\adcme\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\dbeli\.julia\adcme\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: U

(PyObject <tf.Tensor 'add_8:0' shape=() dtype=float64>, PyObject <tf.Tensor 'Mul_17:0' shape=() dtype=float64>, PyObject <tf.Operation 'Adam' type=NoOp>, PyObject <tensorflow.contrib.opt.python.training.external_optimizer.ScipyOptimizerInterface object at 0x00000000016F6788>, PyObject <tf.Operation 'Adam_1' type=NoOp>, PyObject <tensorflow.contrib.opt.python.training.external_optimizer.ScipyOptimizerInterface object at 0x00000000A4F0AC88>, PyObject <tf.Tensor 'Max_1:0' shape=() dtype=float64>, PyObject <tf.Variable 'Variable_3:0' shape=(1, 64) dtype=float64_ref>, PyObject <tf.Tensor 'Softmax:0' shape=(1, 64) dtype=float64>, PyObject <tensorflow.contrib.opt.python.training.external_optimizer.ScipyOptimizerInterface object at 0x00000000A5110B88>)

In [6]:
sess = Session();
init(sess);


2022-05-11 16:10:00.933108: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2
2022-05-11 16:10:00.940355: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library nvcuda.dll
2022-05-11 16:10:00.984633: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1650 major: 7 minor: 5 memoryClockRate(GHz): 1.56
pciBusID: 0000:01:00.0
2022-05-11 16:10:00.984674: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_100.dll
2022-05-11 16:10:00.989538: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cublas64_100.dll
2022-05-11 16:10:00.994831: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cufft64_100.dll
2022-05-11 16:10:00.997410: I tensorflow/stream_

In [7]:
check = aux_functions.IC()

PyObject <tf.Tensor 'Mul_26:0' shape=(64, 484) dtype=float64>

In [12]:
function P(h_, i, j)
    # get the head for specifi i and j
    tf.strided_slice(h_, [0, i - 1 + (j + 1) * model_param.N_y], [model_param.N_k, (j - 1) * model_param.N_y - 1], [1, model_param.N_y])
end

function P_y(h_, i, j)
    # get the head for specifi i and j
    tf.strided_slice(h_, [0, (i - 1) * model_param.N_x + 1 + j], [model_param.N_k, i * model_param.N_x + j - 1], [1, 1])
end


P_y (generic function with 1 method)

In [18]:
reshape(run(sess, check[1]), (22, 22))[1,:]

22-element Vector{Float64}:
 1.3370153117200534
 1.326121253111723
 1.3148082433966222
 1.3029731681222065
 1.2905048042820204
 1.2772951874122507
 1.2632581974807224
 1.248354766271721
 1.2326203684947719
 1.2161858989373335
 1.1992805728777187
 1.1822088510546698
 1.165303524995055
 1.1488690554376166
 1.1331346576606676
 1.118231226451666
 1.1041942365201376
 1.090984619650368
 1.078516255810182
 1.066681180535766
 1.0553681708206655
 1.044474112212335

In [92]:
reshape(run(sess, check[1]), (21, 21))[ 9:12,:]

4×21 Matrix{Float64}:
 0.802209  0.79534   0.78819   0.780688  …  0.640704  0.633554  0.626684
 0.735358  0.729061  0.722508  0.71563      0.587312  0.580758  0.574461
 0.668508  0.662783  0.656825  0.650573     0.53392   0.527962  0.522237
 0.601657  0.596505  0.591143  0.585516     0.480528  0.475165  0.470013

In [46]:
reshape(run(sess,check[1]),(21,21))[1,:][11]

1.1907447119661942

In [19]:
run(sess, P_y(check, 12, -1)[0])

20-element Vector{Float64}:
 1.1822088510546698
 1.125913191480638
 1.069617531906606
 1.013321872332574
 0.9570262127585423
 0.9007305531845103
 0.8444348936104784
 0.7881392340364466
 0.7318435744624147
 0.6755479148883827
 0.6192522553143509
 0.5629565957403189
 0.506660936166287
 0.4503652765922551
 0.3940696170182233
 0.33777395744419136
 0.28147829787015954
 0.2251826382961276
 0.16888697872209568
 0.11259131914806386

In [7]:
run(sess, h_t, feed_dict=Dict( tf_variables.N_k_dis => 4))

50×64×441 Array{Float64, 3}:
[:, :, 1] =
 1.33702  1.33702  1.33702  1.33702  …  1.33702  1.33702  1.33702  1.33702
 1.26669  1.26669  1.26669  1.26669     0.0      0.0      0.0      0.0
 1.23246  1.23246  1.23246  1.23246     0.0      0.0      0.0      0.0
 1.2158   1.2158   1.2158   1.2158      0.0      0.0      0.0      0.0
 1.20769  1.20769  1.20769  1.20769     0.0      0.0      0.0      0.0
 1.20374  1.20374  1.20374  1.20374  …  0.0      0.0      0.0      0.0
 1.20182  1.20182  1.20182  1.20182     0.0      0.0      0.0      0.0
 1.20089  1.20089  1.20089  1.20089     0.0      0.0      0.0      0.0
 1.20043  1.20043  1.20043  1.20043     0.0      0.0      0.0      0.0
 1.20021  1.20021  1.20021  1.20021     0.0      0.0      0.0      0.0
 1.2001   1.2001   1.2001   1.2001   …  0.0      0.0      0.0      0.0
 1.20005  1.20005  1.20005  1.20005     0.0      0.0      0.0      0.0
 1.20002  1.20002  1.20002  1.20002     0.0      0.0      0.0      0.0
 ⋮                              

In [9]:
run(sess, q_t_y, feed_dict=Dict( tf_variables.N_k_dis => 64))

50×64×1 Array{Float64, 3}:
[:, :, 1] =
 1.78446e-9   1.78886e-9   1.78672e-9   …  1.7896e-9    1.78526e-9
 1.49676e-9   1.49714e-9   1.49712e-9      1.50081e-9   1.49766e-9
 1.26821e-9   1.26926e-9   1.27041e-9      1.26941e-9   1.27005e-9
 1.09517e-9   1.09504e-9   1.09264e-9      1.09418e-9   1.09342e-9
 9.65679e-10  9.65896e-10  9.65179e-10     9.6657e-10   9.66359e-10
 8.77985e-10  8.76135e-10  8.78389e-10  …  8.75583e-10  8.78554e-10
 8.14132e-10  8.15459e-10  8.14342e-10     8.14065e-10  8.1502e-10
 7.73214e-10  7.7307e-10   7.73864e-10     7.72271e-10  7.71931e-10
 7.43292e-10  7.42837e-10  7.44792e-10     7.43082e-10  7.42776e-10
 7.24654e-10  7.25112e-10  7.23751e-10     7.25066e-10  7.23491e-10
 7.09857e-10  7.09883e-10  7.11603e-10  …  7.11277e-10  7.10745e-10
 7.01673e-10  7.02509e-10  7.01026e-10     7.00794e-10  7.01733e-10
 6.93937e-10  6.94667e-10  6.94319e-10     6.9557e-10   6.94856e-10
 ⋮                                      ⋱               
 6.80771e-10  6.81679e-10

In [21]:
loss, dw_2, opt_ADAM, opt_LFGS, opt_ADAM_sum, opt_LFGS_sum, diff_eval, p_pre_soft_max, p, opt_LFGS_x = Info_upscale(tf_variables, param_model_val, q_t_x, N_k_dis_, maxiter)


(PyObject <tf.Tensor 'Mul_23:0' shape=() dtype=float64>, PyObject <tf.Tensor 'Mul_21:0' shape=() dtype=float64>, PyObject <tf.Operation 'Adam' type=NoOp>, PyObject <tensorflow.contrib.opt.python.training.external_optimizer.ScipyOptimizerInterface object at 0x00000000A2514748>, PyObject <tf.Operation 'Adam_1' type=NoOp>, PyObject <tensorflow.contrib.opt.python.training.external_optimizer.ScipyOptimizerInterface object at 0x000000018E65E308>, PyObject <tf.Tensor 'Max_1:0' shape=() dtype=float64>, PyObject <tf.Variable 'Variable_6:0' shape=(1, 64) dtype=float64_ref>, PyObject <tf.Tensor 'Softmax:0' shape=(1, 64) dtype=float64>, PyObject <tensorflow.contrib.opt.python.training.external_optimizer.ScipyOptimizerInterface object at 0x000000018E6D11C8>)